In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# Window functions¶
-  `pyspark.sql.functions`는 substr()과 같이 문자열을 잘라내고, 그 결과를 생성하여 행으로 적용
- 윈도우 함수는 그룹으로 구분하고 그 범위 내에서 계산을 할 때 사용, 이 때 `over()` 함수로 적용되는 그룹의 윈도우를 구분하게 된다.
    - 순위 함수 ranking functions: rank, dense_rank, percent_rank, ntile, row_number
    - 분석 함수 analytic functions: cume_dist, first_value, last_value, lag, lead
    - 집합 함수 aggregate functions: sum, avg, min, max, count와 같이 앞서 배웠던 집합 함수를 윈도우에 대해 적용.

In [5]:
#list 안에 문자열로 되어있다.
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

### df로
- 이름, 과목, 성적을 하나의 컬럼으로 읽음 -> 이를 컴마로 분리하여 하나의 컬럼 생성


- RDD의 map()을 사용하여 컴마로 분할 -> 3개의 컬럼으로 dataframe 생성

In [6]:
# RDD로 만들어서 ,로 분리하자
# x: "김하나, English, 100"
# space로 분리되어 있을 경우 아무것도 안적어도 된다.
_marksRdd=spark.sparkContext.parallelize(marks).map(lambda x:x.split(','))

In [7]:
#읽으면서 컬럼명 수정
_marksDf=spark.createDataFrame(_marksRdd, schema=["name", "subject", "mark"])

In [8]:
_marksDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)



In [9]:
_marksDf.show()

+------+--------+-----+
|  name| subject| mark|
+------+--------+-----+
|김하나| English|  100|
|김하나|    Math|   80|
|임하나| English|   70|
|임하나|    Math|  100|
|김갑돌| English| 82.3|
|김갑돌|    Math| 98.5|
+------+--------+-----+



In [10]:
_marksDf.groupBy('subject').count().show()

+--------+-----+
| subject|count|
+--------+-----+
|    Math|    3|
| English|    3|
+--------+-----+



###  group window (그룹 window)
- `partitionBy`: 컬럼별로 구분, 즉 컬럼 값에 따라 partition에 포함되는 행을 할당.
    - partition을 정하지 않으면 모든 행을 동일한 노드에 할당한다.
- `orderBy`: partition 내에서 컬럼 값에 대해 행의 순서를 정렬한다. 순서대로 정렬
- `frame`: 현재 행을 기준으로 포함할 행을 분할.
    - 행 프레임: 현재 행을 기준으로 몇 개 앞, 몇 개 뒤의 물리적 범위를 `window.rowsBetween(start, end)` 정한다.
    - 범위 프레임: 논리적인 범위를 `windowSpec.rangeBetween(start, end)` 정한다. 현재 성적이 70점이면 RANGE BETWEEN 20 PRECEDING AND 10 FOLLOWING은 50 ~ 80을 의미(앞에 20 뒤에 10)

In [11]:
from pyspark.sql.window import Window

win = Window.partitionBy("subject").orderBy("mark") #과목별로

### 순위 함수

#### row_number
`row_number()` 윈도우 함수는 각 그룹별로 일련번호를 생성한다.

In [12]:
from pyspark.sql.functions import row_number
_marksDf.withColumn("row_number", row_number().over(win)).show()
#over안에 위에서 만든 win 적는다.

+------+--------+-----+----------+
|  name| subject| mark|row_number|
+------+--------+-----+----------+
|임하나|    Math|  100|         1|
|김하나|    Math|   80|         2|
|김갑돌|    Math| 98.5|         3|
|김하나| English|  100|         1|
|임하나| English|   70|         2|
|김갑돌| English| 82.3|         3|
+------+--------+-----+----------+



- 정렬이 되지 않고 있다. ==> `mark`가 문자열로 정의되어있기 때문

In [14]:
_marksDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)



In [16]:
# FloatType()로 형변환
from pyspark.sql.types import FloatType
_marksDf = _marksDf.withColumn('markF', _marksDf['mark'].cast(FloatType()))
# markF라는 컬럼을 만들고 cast해서 FloatType로

In [18]:
from pyspark.sql import functions as F
winF = Window.partitionBy("subject").orderBy(F.col("markF").desc())
# Float로 winF다시만듦

In [19]:
from pyspark.sql.functions import row_number
_marksDf.withColumn("row_number", row_number().over(winF)).show()

+------+--------+-----+-----+----------+
|  name| subject| mark|markF|row_number|
+------+--------+-----+-----+----------+
|임하나|    Math|  100|100.0|         1|
|김갑돌|    Math| 98.5| 98.5|         2|
|김하나|    Math|   80| 80.0|         3|
|김하나| English|  100|100.0|         1|
|김갑돌| English| 82.3| 82.3|         2|
|임하나| English|   70| 70.0|         3|
+------+--------+-----+-----+----------+



#### rank
`rank()` 윈도우 함수는 각 그룹별로 등위를 계산한다.

In [27]:
from pyspark.sql.functions import rank
_marksDf.withColumn("rank", rank().over(winF)).show()

+------+--------+-----+-----+----+
|  name| subject| mark|markF|rank|
+------+--------+-----+-----+----+
|임하나|    Math|  100|100.0|   1|
|김갑돌|    Math| 98.5| 98.5|   2|
|김하나|    Math|   80| 80.0|   3|
|김하나| English|  100|100.0|   1|
|김갑돌| English| 82.3| 82.3|   2|
|임하나| English|   70| 70.0|   3|
+------+--------+-----+-----+----+



### 분석적 함수

#### cume_dist()
`cume_dist()` 윈도우 함수는 누적 분포 값을 출력한다.

In [21]:
from pyspark.sql.functions import cume_dist

_marksDf.withColumn("cume_dist", cume_dist().over(winF)).show()

+------+--------+-----+-----+------------------+
|  name| subject| mark|markF|         cume_dist|
+------+--------+-----+-----+------------------+
|임하나|    Math|  100|100.0|0.3333333333333333|
|김갑돌|    Math| 98.5| 98.5|0.6666666666666666|
|김하나|    Math|   80| 80.0|               1.0|
|김하나| English|  100|100.0|0.3333333333333333|
|김갑돌| English| 82.3| 82.3|0.6666666666666666|
|임하나| English|   70| 70.0|               1.0|
+------+--------+-----+-----+------------------+



In [22]:
from pyspark.sql.functions import lag
# lag는 하나씩 밀려서 나온다
_marksDf.withColumn("lag", lag('mark', 1).over(winF)).show()

+------+--------+-----+-----+-----+
|  name| subject| mark|markF|  lag|
+------+--------+-----+-----+-----+
|임하나|    Math|  100|100.0| null|
|김갑돌|    Math| 98.5| 98.5|  100|
|김하나|    Math|   80| 80.0| 98.5|
|김하나| English|  100|100.0| null|
|김갑돌| English| 82.3| 82.3|  100|
|임하나| English|   70| 70.0| 82.3|
+------+--------+-----+-----+-----+



In [23]:
from pyspark.sql.functions import lead
#1개씩 앞당김
_marksDf.withColumn("lag", lead('mark', 1).over(winF)).show()

+------+--------+-----+-----+-----+
|  name| subject| mark|markF|  lag|
+------+--------+-----+-----+-----+
|임하나|    Math|  100|100.0| 98.5|
|김갑돌|    Math| 98.5| 98.5|   80|
|김하나|    Math|   80| 80.0| null|
|김하나| English|  100|100.0| 82.3|
|김갑돌| English| 82.3| 82.3|   70|
|임하나| English|   70| 70.0| null|
+------+--------+-----+-----+-----+



### Aggregate Functions
그룹을 구분할 때, 정렬할 필요가 없다.  
mark가 string으로 설정되어 있다고 평균, 합계, 최소, 최대 함수가 어떻게 출력될까? 문제 없이 실행되지만, 결과는 올바르지 않다. **데이터타입은 항상 주의**해야 한다

In [24]:
winAgg  = Window.partitionBy("subject")

In [25]:
from pyspark.sql import functions as F
_marksDf.withColumn("avg", F.avg(F.col("markF")).over(winAgg))\
    .withColumn("sum", F.sum(F.col("markF")).over(winAgg))\
    .withColumn("min", F.min(F.col("markF")).over(winAgg))\
    .withColumn("max", F.max(F.col("markF")).over(winAgg))\
    .show() #수학,과목끼리, 영어과목끼리 같은 avg

+------+--------+-----+-----+-----------------+-----------------+----+-----+
|  name| subject| mark|markF|              avg|              sum| min|  max|
+------+--------+-----+-----+-----------------+-----------------+----+-----+
|김하나|    Math|   80| 80.0|92.83333333333333|            278.5|80.0|100.0|
|임하나|    Math|  100|100.0|92.83333333333333|            278.5|80.0|100.0|
|김갑돌|    Math| 98.5| 98.5|92.83333333333333|            278.5|80.0|100.0|
|김하나| English|  100|100.0|84.10000101725261|252.3000030517578|70.0|100.0|
|임하나| English|   70| 70.0|84.10000101725261|252.3000030517578|70.0|100.0|
|김갑돌| English| 82.3| 82.3|84.10000101725261|252.3000030517578|70.0|100.0|
+------+--------+-----+-----+-----------------+-----------------+----+-----+

